## Lesson 22

Note: For this lesson I am just making rough notes. This lecture is a continuation of bags of tricks for training diffusion models.

### Cosine Schedular

Notebook [22_cosine.ipynb](https://github.com/fastai/course22p2/blob/master/nbs/22_cosine.ipynb) uses the cosine schedule but made some changes: 

- Got rid of capital T, instead time goes from 0 to 1 , and we calculate abar from that.
- Uses cosine schedule to calculate abar from time (x)
- Added an inverse to calculate t from x (abar)
- This changes noisify to use the function instead of the list. (see notebook for details, I didn't reproduce it here.)
- everything else is the same as far as fitting etc. 
- Sampling now uses linspace to step through the schedule (0 to 1) for 'steps' steps.
- The iamges do look better?  Not clear.


### Predicting Noise Level 
10:56 in the video

Notebook [22_noise-pred.ipynb](https://github.com/fastai/course22p2/blob/master/nbs/22_noise-pred.ipynb) 

####  Noise prediction model
- Different kind of model.. create a model that predicts 't' given the noisy image.
- Why ? He wanted to understand why we need to pass in t as well as the noised image. Can the model figure out how much noise there is? Is this side information needed?
- Sets $\bar{\alpha}_t$ as the target, and so he sets it to a random number from zero to 1 (and then takes the logit to map to the real line) as the actual target. This makes give equal weight to ratios at both ends of the scale.
- For comparison, he has a baseline model that always predicts 0 ( i.e. logit(0.5))  Baseline loss with that can be compared.
- He uses basically the same model except with a 1 output on the output layer.  
- THe model works really well at predicting the amount of noise, which was Jeremies hypothesis. This seems to justify not passing in t.

#### No-t model
- He then tries a diffusion model that doesn't take in t during training.
- Basically the same code from the previous notebook but remove t from the input to the unet. (he just passes in 0) The loss was maybe a tiny bit worse, but not much.
- For sampling, he uses the same ddim step, except we can't pass in t to the model, because it doesn't take  it. 
- Results, not good ('garbage').  Fid was 22 (vs. 3.3 for the model that took in t in the previous notebook 22_cosine.ipynb)
- First fix he noticed that the ddim formula was still using the abar calculated from the t in the step, so instead he used abar estimated using the noise prediction model, except he also clamped it so that it would not be too crazy (using the median of the abars in the batch... `clamp(med/2, 2*med)`). He used this abar in the ddim formula.
- THis was a huge improvement , fid went down to 3.9.  
- This fix was after multiple days of discussion and debugging.  Just like with the DDPM bug...

### Back to DDPM 'bug'. 
At 30:36 in video

- Remember that the DDPM images were not scaled from -1 to 1 and maybe it was not such a good idea to do that anyway. (he ended up going from -0.5 to 0.5) 
- Paper came out that casts some light on this: [On the Importance of Noise Scheduling for Diffusion Models](https://arxiv.org/abs/2301.10972)  
- Shows Noise scheduling is crucial and depends on the size of the images. (This makes sense, as for higher resolution, neihboring pixes are more correlated)
- Considers different noise schedules (e.g. Cosine, sigmoid)
- Scaling images are important as well. Adjusting the input scaling can improve the training, since this changes the ratio of signal to noise. So this is what was accidently going on with the -1 to 1 scaling.




### Karras et. al 2022
38:45 in video.

- [Notebook 23_karras.ipynb](https://github.com/fastai/course22p2/blob/master/nbs/23_karras.ipynb
- Paper : [Elucidating the Design Space of Diffusion-based Generative Models](https://arxiv.org/abs/2206.00364)

#### Training targets and scaling
- In this notebook, they return to the -1 to 1 scaling, and will rely on Karras paper to scale the images.  
- LOL despite all that he found another factor of two bug where the bug works better (usign std of 0.33 when it is really 0.66).
- In Karras, instead of having the noise as the training target, they predict the noise, the  image, or somewhere between the two depending on the noise. (Idea is that if the image is mostly noise, then predicting noise is a waste of time, instead try to predict the image). (Also called v-objective?)
- The paper also looked at the loss as a function of sigma and found that noise in the middle was the best, so they used a lognormal distributino of sigma values to focus the targets in the middle range. (see paper). This will be used by the new noisify.   They also give formula for  scalign the image and the noise for the training targets. (For example to scale the inputs to have unit variance, so divide by the total standard deviation)
- **NOTE:** I think the offsetting bug is that they use 1/variance instead of 1/std for scaling in the notebook, which would explain why 0.33 works better then 0.66. (Sqrt of 0.33 is 0.57, which is close to 0.66)
- The training is essentially the same as before, we see different losses of course.
- Denoising is the same, just have to use c_skip and c_out .  It is quite remarkable that the denoising works so well (this is single step denoise, not iterative samplign).

#### Sampling Kerras
- Karras ran a lot of experiments on sampling, and found a better way to step through the noise rather then linearlly.  They used an exponential function and tried different values for the exponent and ended up using 1/7.  
- The sampler was derivied from an SDE and so uses an SDE solver, the simplest of which is the Euler method.  This part was a bit confusing in the lecture and I would really need to spend time with the paper to grok this, and I just am not going to right now. (The paper   used Heun's 2nd order method, but does mention Euler)
- With Euler, FID is 1.98 which is the best so far. 
- The also add noise into the sampling (the EUler didnt add noise) and this improved the FID to 1.53
- With Heun's 2nd order method (another method similar to Runge-Kutta)   FID was 0.97 (with 50 steps = 100 model evaluations). 
- He then talks about lms sampler whatever that is.

### Preview of next lessson

- Found a bug in mearuing FID (another issue with scaling). When he fixed teh FID's weere much bigger, in 5-6 range. Cosine was 3-4 so these are not so much better after all!!

This section ends at 6 minutes in.
